In [ ]:
from nilearn.maskers import NiftiLabelsMasker
from scipy.ndimage import center_of_mass
import nibabel as nib
import numpy as np
import pandas as pd
import os

In [ ]:
# Define data directory and mapping file
data_dir = '/home/zaz22/research-data-store/fmri/fmri_beijing'
mapping_file = '/home/zaz22/research-data-store/rois/rois_3000_beijing/rois/brain_atoms.mnc.gz'  

# Define repetition time (TR)
repetition_time = 2.0

# List of subject IDs
# subject_ids = ['9640133', '9783279', '9887336', '9890726', '4095748', '4136226', '4221029',
#                 '4225073', '4241194', '4256491', '4265987', '4334113', '4383707', '4475709',
#                 '4921428', '5150328', '5193577', '5575344', '5600820', '5669389', '5993008',
#                 '6187322', '6383713', '6477085', '6500128', '7011503', '7093319', '7135128',
#                 '7253183', '7390867', '7407032', '7689953', '7994085', '8191384', '8278680',]

# subject_ids = [
#     '9640133', '9783279', '9887336', '9890726', '4095748', '4136226', '4221029',
#     '4225073', '4241194', '4256491', '4265987', '4334113', '4383707', '4475709',
#     '4921428', '5150328', '5193577', '5575344', '5600820', '5669389', '5993008',
#     '6187322', '6383713', '6477085', '6500128', '7011503', '7093319', '7135128',
#     '7253183', '7390867', '7407032', '7689953', '7994085', '8191384', '8278680',
#     '8328877', '8838009', '9002207', '9093997', '9210521', '9221927', '9578631',
#     '3494778', '3554582', '3561920', '3562883', '3587000', '3593327', '3610134',
#     '3624598', '3655623', '3672300', '3672854', '3691107', '3707771', '3712305',
#     '3732101', '3739175', '3803759', '3809753', '3827352', '3856956', '3870624',
#     '3889095', '3910672', '3930512', '3967265', '3976121', '3983607', '3993793',
#     '3994098', '4006710', '4028266', '4048810', '4053388', '4053836', '4055710',
#     '4073815', '4075719', '4091983', '2538839', '2559537', '2601519', '2659769',
#     '2697768', '2703336', '2714224', '2737106', '2780647', '2833684', '2884672',
#     '2897046', '2907951', '2910270', '2919220', '2940712', '2950754', '2984158',
#     '3004580', '3086074', '3157406', '3194757', '3205761', '3212536', '3224401',
#     '3233028', '3239413', '3248920', '3262042', '3269608', '3277313', '3291029',
#     '3306863', '3308331', '3385520', '3390312', '3446674', '3473830', '1809715',
#     '1843546', '1860323', '1875013', '1875711', '1879542', '1912810', '1916266',
#     '1947991', '2031422', '2033178', '2081754', '2106109', '2107404', '2123983',
#     '2140063', '2141250', '2174595', '2196753', '2207418', '2208591', '2228148',
#     '2240562', '2249443', '2266806', '2268253', '2276801', '2296326', '2310449',
#     '2367157', '2377207', '2408774', '2427408', '2493190', '2498847', '2524687',
#     '2529026', '2535087', '1050345', '1050975', '1056121', '1068505', '1093743',
#     '1094669', '1113498', '1117299', '1132854', '1133221', '1139030', '1159908',
#     '1177160', '1186237', '1240299', '1258069', '1282248', '1302449', '1341865',
#     '1356553', '1391181', '1399863', '1404738', '1408093', '1411536', '1469171',
#     '1494102', '1561488', '1562298', '1628610', '1643780', '1662160', '1686092',
#     '1689948', '1771270', '1791543', '1794770', '1805037'
# ]

subject_ids = [
    '9640133', '9783279', '9887336', '9890726', '4095748', '4136226', '4221029',
    '4225073', '4241194', '4256491', '4265987', '4334113', '4383707', '4475709',
    '4921428', '5150328', '5193577', '5575344', '5600820', '5669389', '5993008',
    '6187322', '6383713', '6477085', '6500128', '7011503', '7093319', '7135128',
    '7253183', '7390867', '7407032', '7689953', '7994085', '8191384', '8278680',
    '8328877', '8838009', '9002207', '9093997', '9210521', '9221927', '9578631',
    '3494778', '3554582', '3561920', '3562883', '3587000', '3593327', '3610134',
    '3624598', '3655623', '3672300', '3672854', '3691107', '3707771', '3712305',
    '3732101', '3739175', '3803759', '3809753', '3827352', '3856956', '3870624',
    '3889095', '3910672', '3930512', '3967265', '3976121', '3983607', '3993793',
    '3994098', '4006710', '4028266', '4048810', '4053388', '4053836', '4055710',
    '4073815', '4075719', '4091983', '2538839', '2559537', '2601519', '2659769',
    '2697768', '2703336', '2714224', '2737106', '2780647', '2833684', '2884672',
    '2897046', '2907951', '2910270', '2919220', '2940712', '2950754', '2984158',
    '3004580', '3086074', '3157406', '3194757', '3205761', '3212536', '3224401'
]


In [ ]:
# Load the ROI mapping file
mapping_img = nib.load(mapping_file)
mapping_data = mapping_img.get_fdata()
affine = mapping_img.affine

# Get unique ROI labels, excluding 0
roi_labels = np.unique(mapping_data)
roi_labels = roi_labels[roi_labels != 0]
print(f'Number of ROIs: {len(roi_labels)}')


In [ ]:
# Extract ROI coordinates (center of mass)
roi_coords = []

for label in roi_labels:
    # Create a binary mask for the current ROI
    roi_mask = mapping_data == label

    # Calculate the center of mass in voxel space
    com_voxel = center_of_mass(roi_mask)

    # Convert voxel coordinates to world coordinates
    com_world = nib.affines.apply_affine(affine, com_voxel)

    roi_coords.append(com_world)

print(f'ROI coordinates (world space): {roi_coords}')


In [ ]:
# Initialize a masker for extracting time series from ROIs
labels_masker = NiftiLabelsMasker(
    labels_img=mapping_img,
    standardize=True,
    detrend=True,
    t_r=repetition_time,
    verbose=1
)

# Initialize a list to store time series for all subjects
all_tseries = []

for sub_id in subject_ids:
    # Build the functional file path
    func_file = os.path.join(data_dir, f'fmri_X_{sub_id}_session_1_run1.nii.gz')
    print(f'Loading functional image for subject {sub_id} from {func_file}')
    
    # Load the functional image
    func_img = nib.load(func_file)
    
    # Extract time series using the labels masker
    tseries = labels_masker.fit_transform(func_img)
    
    all_tseries.append(tseries)
    print(f'Extracted time series for subject {sub_id}, shape: {tseries.shape}')


In [ ]:
# Define file paths and parameters
behavioral_file = '/home/zaz22/repos/adhd-mri-using-ml/adhd200_preprocessed_phenotypics.tsv'  # Update with your actual file path
behavioral = pd.read_csv(behavioral_file, sep='\t')

# Define diagnosis groups
behavioral['DX'] = behavioral['DX'].astype(str).str.strip()

# Apply mapping
conditions_map = {
    '0': 'Non-ADHD',
    '1': 'ADHD',
    '2': 'ADHD',
    '3': 'ADHD'
}
behavioral['Diagnosis_Class'] = behavioral['DX'].map(conditions_map)
print(behavioral['Diagnosis_Class'].value_counts(dropna=False))


In [ ]:
# Map the diagnosis column to binary classes
behavioral['Diagnosis_Class'] = behavioral['DX'].map(conditions_map)
behavioral = behavioral.loc[behavioral['DX'] != 'pending']
print(behavioral['DX'].unique())  # Check all unique values in the DX column
print(behavioral['Diagnosis_Class'].value_counts())  # Check how the mapping works
print(behavioral[behavioral['DX'].isna()])  # Check rows where DX is NaN
print(behavioral['Diagnosis_Class'].unique())  # Check unique mapped values
print(behavioral['DX'].dtypes)  # Check if the column is numeric or string
print(behavioral['DX'].unique())  # Confirm all unique values


In [ ]:
# Filter data for valid diagnosis classes
condition_mask = behavioral['Diagnosis_Class'].notna()
print(f"Condition mask shape: {condition_mask.shape}")
print(condition_mask.sum())  # Check how many rows are being selected

filtered_behavioral_data = behavioral[condition_mask]
print(f"Behavioral data shape after filtering: {filtered_behavioral_data.shape}")

# Extract valid subject IDs that match 'subject_ids'
valid_subject_ids = [
    sub_id for sub_id in subject_ids if sub_id in filtered_behavioral_data['ScanDir ID'].astype(str).tolist()
]
print(f"Valid Subject IDs: {valid_subject_ids}")

# Filter fMRI images using the valid subject IDs
valid_fmri_imgs = []

for subject_id in valid_subject_ids:
    # Build the path to the subject's fMRI file
    fmri_path = os.path.join(data_dir, f'fmri_X_{subject_id}_session_1_run1.nii.gz')

    # Check if the file exists before attempting to load
    if not os.path.exists(fmri_path):
        print(f"File not found for Subject ID: {subject_id}")
        continue

    # Load the subject's fMRI image
    fmri_img = nib.load(fmri_path)
    valid_fmri_imgs.append(fmri_img)

# Function to check and correct shapes
from nilearn.image import resample_img, concat_imgs

def check_and_correct_shapes(image_list):
    if not image_list:
        raise ValueError("The image list is empty.")
    
    # Reference shape and affine from the first image
    reference_img = image_list[0]
    target_shape = reference_img.shape[:3]  # Only spatial dimensions
    target_affine = reference_img.affine

    corrected_images = []
    for idx, img in enumerate(image_list):
        if img.shape[:3] != target_shape:
            print(f"Resampling image {idx} to match target shape {target_shape}")
            img = resample_img(img, target_affine=target_affine, target_shape=target_shape)
        corrected_images.append(img)
    
    return corrected_images

# Check and correct fMRI image shapes
try:
    valid_fmri_imgs = check_and_correct_shapes(valid_fmri_imgs)
    # Concatenate all valid fMRI images into a single 4D image
    fmri_imgs = concat_imgs(valid_fmri_imgs)
    print(f"Concatenated fMRI images shape: {fmri_imgs.shape}")
except Exception as e:
    print(f"Error during processing: {e}")


In [ ]:
from nilearn.image import index_img
from sklearn.metrics import accuracy_score
from nilearn.decoding import Decoder
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.linear_model import LogisticRegression

# Ensure fmri_imgs and valid_subject_ids are already defined
print(f"fMRI data shape: {fmri_imgs.shape}")

# Extract the relevant conditions
conditions = filtered_behavioral_data['Diagnosis_Class'].values
print(f"Conditions shape: {conditions.shape}")


In [ ]:

# Initialize the decoder with desired parameters

# REGRESSION: 
# For regression, valid entries are: ‘r2’, ‘neg_mean_absolute_error’, or ‘neg_mean_squared_error’.

        # DEFAULT DECODER:
        # Use when you have a small-to-medium dataset with potentially non-linear decision boundaries.
        # Great for clean data with clearly separable classes.
#decoder = Decoder(estimator="svc", standardize="zscore_sample", cv=5, scoring="accuracy",n_jobs=16)

        # L2 DECODER:
        # l2 regularization penalizes the sum of the squares of the weights.
        # L2 regularization keeps all features but reduces their influence
# decoder = Decoder(estimator="logistic_l2", standardize="zscore_sample", cv=5, scoring="accuracy",n_jobs=16)

        # L1 DECODER: 
        # L1 regularization reduces model complexity by eliminating some features, 
        # L1 regularization penalizes the sum of the absolute values of the weights
# logistic_l1 = LogisticRegression(penalty="l1", solver="liblinear", max_iter=10000)
decoder = Decoder(estimator="logistic_l1", standardize="zscore_sample", cv=10, scoring="accuracy",n_jobs=16)


In [ ]:
# print("decoder.cv_scores_ keys:", decoder.cv_scores_.keys())
# print("decoder.cv_scores_ content:", decoder.cv_scores_)


In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_accuracies = []
training_losses = []
validation_losses = []

for fold, (train_idx, test_idx) in enumerate(kf.split(conditions), start=1):
    # Split the data into training and test sets
    fmri_train = index_img(fmri_imgs, train_idx)
    fmri_test = index_img(fmri_imgs, test_idx)
    conditions_train = conditions[train_idx]
    conditions_test = conditions[test_idx]

    # Train the decoder
    decoder.fit(fmri_train, conditions_train)

    # Calculate training loss
    # Average scores across all labels and folds
    mean_train_score = np.mean([
        np.mean(scores) for scores in decoder.cv_scores_.values()
    ])
    train_loss = 1 - mean_train_score
    training_losses.append(train_loss)
    
    # Predict on the test set
    predictions = decoder.predict(fmri_test)
    
    # Calculate validation loss (1 - test accuracy)
    val_loss = 1 - accuracy_score(conditions_test, predictions)
    validation_losses.append(val_loss)

    # Calculate and store fold accuracy
    accuracy = accuracy_score(conditions_test, predictions)
    fold_accuracies.append(accuracy)
    print(f"Fold {fold}: Accuracy = {accuracy:.2f}, Training Loss = {train_loss:.2f}, Validation Loss = {val_loss:.2f}")

# Report overall accuracy
mean_accuracy = np.mean(fold_accuracies)
print(f"Mean Cross-Validation Accuracy: {mean_accuracy:.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Plotting cross-validation accuracies
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(training_losses) + 1), training_losses, label='Training Loss', marker='o')
plt.plot(range(1, len(validation_losses) + 1), validation_losses, label='Validation Loss', marker='o')
plt.xlabel('Fold')
plt.ylabel('Loss')
plt.title('Training vs. Validation Loss Across Folds')
plt.legend()
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(fold_accuracies) + 1), fold_accuracies, label='Fold Accuracy', marker='o', color='green')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Across Folds')
plt.legend()
plt.grid()
plt.show()